# 准备
### 内存优化
如果直接复用现有变量存储Y = X + Y，会浪费一次内存分配。 我们也可以使用X[:] = X + Y或X += Y来减少操作的内存开销。



In [ ]:
import torch
X = torch.Tensor([1,2,3])
Y = torch.Tensor([4,5,6])
before = id(X)
X += Y
id(X) == before

### 张量的常规操作


In [ ]:
import torch
# 固定随机种子：让 rand/randn 等随机初始化可复现（不同设备/版本仍可能有细微差异）
torch.manual_seed(42)  
# 选择计算设备：优先 Apple MPS，其次 CUDA GPU，否则 CPU；对比硬编码 'cuda' 更鲁棒
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")  
device  # 在 notebook 中显示当前 device，便于确认张量将被放在哪个设备上


In [ ]:
# 标量和向量
a = torch.tensor([1,2]) ## 1维张量
b = a
s = torch.tensor(1) ## 创建一个0维张量，标量

# 两个向量结算乘法,matmul, @ , dot等价，结果都是内积，也不用转置
c = a.matmul(b) # 两个1维张量计算矩阵乘法，
d = a.dot(b) ## 两个1维张量计算点积，结果是一个标量
e = a @ b ## 两个1维张量计算矩阵乘法，

# 标量可以通过升维变成一个1维张量
s1 = s.unsqueeze(0)
a,b,c,d,e,s,s1

In [ ]:
# 初始化
a = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32, device=device)  # 从 Python 列表创建张量：会拷贝数据；对比 torch.as_tensor/torch.from_numpy（CPU 上）可共享底层内存
b = torch.zeros(2, 3, device=device)  # 全 0 ，第一个参数可以传tuple，也可以直接传分开的参数
c = torch.ones_like(b)  # 形状/类型/设备跟随 b：比 torch.ones(b.shape, device=b.device, dtype=b.dtype) 更不易写错
d = torch.full((2, 2), 3.14, device=device)  # 常数填充，注意第一个参数只能传tuple
e = torch.eye(3, device=device)  # 单位矩阵：常用于线性代数/初始化；对比 torch.diag(torch.ones(n)) 更直接且语义清晰

# 序列类初始化
f = torch.arange(0, 10, step=2, device=device)  # 等差序列：更像 Python range；对比 linspace 用 steps 指定点数而不是步长
g = torch.linspace(0, 1, steps=5, device=device)  # 区间等分采样：适合连续区间；对比 arange 在浮点步长下可能出现累计误差/终点缺失
# ----  # 分隔：随机初始化
h = torch.rand((2, 2), device=device)  # U(0,1) 均匀分布：常用于随机噪声/采样；对比 randn 是 N(0,1) 正态
i = torch.randn((2, 2), device=device)  # N(0,1) 正态分布：常用于权重初始化；对比 normal(mean,std) 可自定义分布参数
j = torch.randint(low=0, high=10, size=(2, 3), device=device)  # 离散均匀整数：常用于类别/索引；对比 rand 需要缩放+取整且分布不精确
# ----  # 分隔：展示结果
a, b, c, d, e  # 返回多个对象让 notebook 一次性展示；对比 print 会把张量转成字符串，后续不便继续链式操作

### 张量的一些骚操作


In [ ]:
# 升维可以使用`unsqueeze`方法，例如`a.unsqueeze(0)`可以在第0维升维，但是也可以用切片+None来升维，例如`a[None, :]`等价于`a.unsqueeze(0)`
# 而且使用切片+None可以一次性升多个维度，而`unsqueeze`方法只能升一个维度
a = torch.arange(5,device=device)
b = a[None, :]  ## 2维张量 等价于c = a.unsqueeze(0)，第0维添加一个维度1，shape:[2]->[1,2]
c = a[None] ## 和上边等价————————第一次看到这个快疯了？这是个啥？哈哈哈哈哈哈哈
d = a[:, None] ## 2维张量，列向量，等价于d = a.unsqueeze(1)，第1维添加一个维度1 shape:[2]->[2,1]
e = a[None,None] ## 一次升2个维度，等价于e = a.unsqueeze(0).unsqueeze(1)，第0维添加一个维度1，第1维添加一个维度1 shape:[2]->[1,1,2]
f = torch.arange(0,12,device=device).reshape(3,4)
g = e[..., None,:]
a,a.shape,a.dim(),b,b.shape,b.dim(),c,c.shape,c.dim(),d,d.shape,d.dim(),e,e.shape,e.dim(),f,f.shape,f.dim(),g,g.shape,g.dim()


#### 1) 初始化（创建张量）
- 从 Python / NumPy 构造：`torch.tensor`
- 常用形状初始化：`zeros/ones/full/empty/eye`
- 序列：`arange/linspace/logspace`，linspace和arange的区别是：linspace的起始值和结束值是固定的，中间的元素是根据步数`step=(end-start)/(steps-1)`算到的；arange是固定起始值，在起始值上增加步长step（linspace 的第三个参数叫steps，arange的第三个参数叫step）
- 随机：`rand/randn/randint/normal`
- `*_like`：保持形状/类型/设备一致：`zeros_like/ones_like/rand_like`


#### 2) 算术运算与广播（Broadcasting）
- 逐元素：`+ - * / **`，`torch.add/sub/mul/div`
- 矩阵乘：`@` / `matmul` / `mm` / `bmm`
- 常用逐元素函数：`exp/log/sqrt/abs/clamp`
- 广播：对齐尾维度，满足相等或为 1


In [ ]:
x = torch.arange(6, device=device, dtype=torch.float32).reshape(2, 3)  # 先生成 0..5，再 reshape 成 2x3；对比 view 只在 contiguous 时可用，reshape 更通用（必要时会拷贝）
y = torch.tensor([1.0, 10.0, 100.0], device=device)  # 1x3 向量：与 x(2x3) 运算会触发广播（broadcasting）
# ----  # 分隔：逐元素算术（elementwise）
elem = x + y  # 逐元素加法 + 广播：语法最简洁；对比 torch.add 更适合需要 out/alpha 或更函数式的写法
scaled = torch.add(x, y, alpha=0.1)  # 计算 x + 0.1*y：一个算子完成缩放+相加；对比先 y*0.1 再加会多一次中间张量分配
powed = x ** 2  # 幂运算：等价 torch.pow(x, 2)；对比 x*x 在指数为 2 时通常更快且更省内存
clamped = torch.clamp(x, min=1.0, max=4.0)  # 截断到区间：[1,4]；对比 relu 只截断下界、hardtanh 类似双边截断
absed = torch.abs(x - y)  # 逐元素取绝对值：等价 torch.abs(x - y)；对比 x-y 直接相减可能得到负数（如 x=2, y=3）
exped = torch.exp(x)  # 逐元素指数：等价 torch.exp(x)；对比 math.exp 只作用于标量
# ----  # 分隔：矩阵乘（matmul）
m1 = torch.randn(2, 3, device=device)  # 随机矩阵：shape(2,3)
m2 = torch.randn(3, 4, device=device)  # 随机矩阵：shape(3,4)
mat = m1 @ m2  # 矩阵乘：等价 torch.matmul(m1, m2)；对比 torch.mm 只支持 2D，matmul 还支持 batch 维度 ## 对比 torch.mm 只支持 2D，matmul 还支持 batch 维度
# mat_alt = m1 * m2  # 逐元素乘：等价 torch.mul(m1, m2)；对比 @ 不支持广播，需手动处理
# ----  # 分隔：爱因斯坦求和（einsum）
eins = torch.einsum('ij,jk->ik', m1, m2)  # 爱因斯坦求和：表达更灵活（可写复杂张量收缩）；对比 @ 更直观，且底层通常更容易走高性能实现
# ----  # 分隔：数值一致性检查
elem, scaled, powed, clamped, (mat - eins).abs().max()  # 比较 matmul 与 einsum 的最大误差：应接近 0（浮点误差除外）


#### 3) 取值、索引与切片
- 基本切片：`x[i]`、`x[:, j]`、`x[..., -1]`
- 布尔掩码：`x[mask]`
- 选取：`index_select`、`gather`（常用于 embedding / attention）
- 原地赋值：切片赋值会直接修改原张量


In [ ]:
x = torch.arange(12, device=device).reshape(3, 4)  # 构造 3x4 矩阵：用连续整数便于直观看出索引位置；对比 randn 不易辨认
x0 = x[0]  # 取第 0 行（shape: 4）；对比 x[0:1] 会保留维度（shape: 1x4），对后续广播更友好
xcol = x[:, 2]  # 取第 2 列（shape: 3）；对比 x[:, 2:3] 会保留列维度（shape: 3x1）
xsub = x[1:, 1:3]  # 切片：行 1..end，列 1..2；切片通常返回视图（view），对比高级索引常返回拷贝
# ----  # 分隔：布尔掩码（mask）
mask = x % 2 == 0  # 生成 bool 掩码：True 表示选中；对比 torch.where 更适合做条件替换/返回坐标
xeven = x[mask]  # 按掩码取值会展平为 1D；对比 masked_select 行为类似但更显式：torch.masked_select(x, mask)
# ----  # 分隔：按索引选取（index_select）
idx = torch.tensor([2, 0], device=device)  # 行索引：要取第 2 行和第 0 行；对比 Python list 索引在 GPU 上不可用
rows = torch.index_select(x, dim=0, index=idx)  # 沿 dim=0 选行；对比 x[idx] 更简洁，但 index_select 在代码审阅时更明确
# ----  # 分隔：gather（逐元素按坐标取值）
src = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], device=device)  # 源矩阵：2x3
picked = torch.gather(src, dim=0, index=torch.tensor([[0, 0, 0], [1, 1, 1]], device=device))  # gather的作用是根据index从src中取出对应元素，组成与index相同shape的tensor
scatter_idx = torch.tensor([[0, 1], [1, 0]], device=device)  # 每个输入位置给出要放的列索引（dim=1）；对比 scatter_nd 只能整体放列
scattered = torch.scatter(src, dim=1, index=scatter_idx, src=picked)  # scatter的作用是根据index将src中的元素放到对应位置，组成与src相同shape的tensor
# ----  # 分隔：展示结果
x0, xcol, xsub, xeven, rows, picked, scattered  # 汇总展示：对比不同索引方式的输出形状与内容


解释gather和scatter的dim和index的关系，假设我们有一个 3D 张量，dim 的不同取值决定了哪个坐标轴被替换：  
如果 dim=0：out[i][j][k] = input[ index[i][j][k] ][j][k]  
如果 dim=1：out[i][j][k] = input[i][ index[i][j][k] ][k]  
如果 dim=2：out[i][j][k] = input[i][j][ index[i][j][k] ]  
也就是说，index的值决定了dim轴上的索引，其他轴上的索引保持不变。

#### 4) 维度变换与拼接
- 形状：`reshape/view/flatten`
- 维度重排：`transpose/permute/movedim`
- 增减维：`unsqueeze/squeeze`
- 拼接与堆叠：`cat/stack`；拆分：`chunk/split`
- 广播扩展：`expand`（不复制数据）、`repeat`（复制数据）


In [ ]:
x = torch.arange(24, device=device).reshape(2, 3, 4)  # 构造 3D 张量：常见形状 (batch, seq, hidden) 或 (N, C, L)
r1 = x.reshape(6, 4)  # 改变形状：必要时会拷贝；对比 view 仅在 contiguous 时可用但通常不拷贝（更轻量）
r2 = x.reshape(2, 3,-1,2)  # 如果某维为 -1，则自动计算该维大小以保持元素总数不变
flat = x.flatten(start_dim=1)  # 从第 1 维开始打平：常用于 (N,C,H,W)->(N,C*H*W)；对比 reshape 更直观、少算维度
# ----  # 分隔：维度重排
t = x.transpose(1, 2)  # 交换两个维度：只支持两维互换；对比 permute 可任意重排多个维度
p = x.permute(2, 0, 1)  # 任意维度重排：更通用；对比 transpose 更简洁但能力更弱
# ----  # 分隔：增减维
u = x.unsqueeze(0)  # 增加长度为 1 的维度：常用于对齐 batch 维或做广播；对比 reshape 插维更容易写错
s = u.squeeze(0)  # 去掉指定的长度为 1 的维度：与 unsqueeze 成对；对比 squeeze() 不指定 dim 可能误删其他 1 维
# ----  # 分隔：拼接/堆叠/拆分
a = torch.zeros((2, 3), device=device)  # 准备张量 a：用于对比 cat 与 stack
b = torch.ones((2, 3), device=device)  # 准备张量 b：与 a 同形状
cat0 = torch.cat([a, b], dim=0)  # 沿已有维度拼接：dim=0 变长 (4,3)；对比 stack 会新增一维
stk0 = torch.stack([a, b], dim=0)  # 新增一维再堆叠：结果 (2,2,3)；对比 cat 更适合把 batch 维拼大
c1, c2 = cat0.chunk(2, dim=0)  # 均匀拆分：把 (4,3) 拆成两个 (2,3)；对比 split 可按指定长度不等分
# ----  # 分隔：广播扩展
base = torch.arange(3, device=device).reshape(1, 3)  # 基础行向量：shape(1,3)
expanded = base.expand(4, 3)  # 视图式扩展：不复制数据（stride 可能为 0）；对比 repeat 会真实复制、占更多内存
repeated = base.repeat(4, 1)  # 数据复制：得到独立内存；对比 expand 适合只读广播，repeat 适合后续需要原地写/独立修改
# ----  # 分隔：展示各操作的形状与一致性
r1.shape,r2.shape, flat.shape, t.shape, p.shape, u.shape, (s == x).all(), cat0.shape, stk0.shape, (c1 + c2).shape, expanded.shape, repeated.shape  # 用 shape 对比各方法对维度的影响


#### 5) 聚合统计与常用数学
- 规约：`sum/mean/max/min`，配合 `dim` 与 `keepdim`
- 位置：`argmax/argmin/topk`
- 累积：`cumsum/cumprod`
- 稳定计算：`logsumexp`、`softmax`


In [ ]:
x = torch.randn(2, 3, 4, device=device)  # 构造 3D 随机张量：用于演示 dim 维度规约
s1 = x.sum(dim=2)  # 沿 dim=2 求和：把最后一维规约掉；对比 sum() 不给 dim 会把所有元素求和得到标量
m1 = x.mean(dim=(1, 2), keepdim=True)  # 多维求均值并保留维度：keepdim 便于后续广播；对比不 keepdim 会少维度导致广播需手动 unsqueeze
mx = x.max(dim=-1).values  # max 返回 (values, indices)：这里只取最大值；对比 torch.amax 只返回值但不返回位置
am = x.argmax(dim=-1)  # 最大值位置：常用于分类预测；对比 argmax 不给 dim 会在展平后找全局最大位置
# ----  # 分隔：top-k
scores = torch.randn(2, 5, device=device)  # 2 个样本，每个 5 个分数（logits）
topv, topi = torch.topk(scores, k=2, dim=-1)  # 取每行最大的 2 个值及其索引；对比 sort 更通用但更慢且会做全量排序
# ----  # 分隔：数值稳定的归一化
ls = torch.logsumexp(scores, dim=-1)  # log(sum(exp(x))) 的稳定写法：对比 log(exp(x).sum()) 容易上溢/下溢
sm = torch.softmax(scores, dim=-1)  # softmax：把 logits 变成概率分布；对比 sigmoid 用于二分类/多标签，不会对全向量做归一
# ----  # 分隔：快速自检
s1.shape, m1.shape, mx.shape, am.shape, topv, topi, ls, sm.sum(dim=-1)  # softmax 每行和应为 1（允许浮点误差）


#### 6) 类型、设备与自动求导
- 类型：`to(dtype=...)`、`float/long/half`
- 设备：`to(device)`、`cpu/cuda`
- 梯度：`requires_grad_()`、`backward()`、`detach()`
- 常见：loss 标量对参数求导，`grad` 存在 `param.grad`


In [ ]:
w = torch.randn(3, 1, device=device, requires_grad=True)  # 参数张量：requires_grad=True 才会在 backward 时累积梯度；对比 w.requires_grad_(True) 是原地设置
x = torch.randn(8, 3, device=device)  # 输入特征：8 个样本、3 维特征；对比把 x 设 requires_grad 多用于需要输入梯度的任务（如对抗样本）
y = torch.randn(8, 1, device=device)  # 目标值：这里用回归形式演示（实际训练多来自数据集）
# ----  # 分隔：前向计算
pred = x @ w  # 线性层前向：等价 torch.matmul(x, w)；对比 torch.nn.Linear 会管理参数与 bias，更适合构建模型
loss = ((pred - y) ** 2).mean()  # MSE 损失：先平方再平均；对比 torch.nn.functional.mse_loss 更标准且支持不同 reduction
loss.backward()  # 反向传播：从 loss 触发计算图求导，并把梯度累加到 w.grad；对比 torch.autograd.grad 返回梯度但不累积
# ----  # 分隔：取标量与查看梯度
loss.item(), w.grad.shape  # item() 转 Python 标量（可能触发设备同步）；对比 loss.detach() 保持张量形态便于批量日志


#### 7) 计算性能与内存优化（常用做法）
- 尽量用向量化/矩阵化替代 Python 循环
- 训练外推理：`torch.no_grad()` 或 `torch.inference_mode()`
- 原地操作减少分配：`x+=y` 等价与 `x[:]=x+y`都会避免一次额外的内存分配，但是如果直接`x=x+y`会导致一次额外的内存分配
- 避免频繁 `cpu()` / `cuda()` 往返，减少小张量 `item()`
- 对 GPU：`pin_memory` + `non_blocking=True` 传输；混合精度 `autocast`；`torch.compile`（可用时）


In [ ]:
x = torch.randn(1024, 1024, device=device)  # 构造大矩阵：便于观察算子是否产生额外内存分配/缓存
y = torch.randn(1024, 1024, device=device)  # 同形状矩阵：用于逐元素加法与矩阵乘
# ----  # 分隔：原地 vs 非原地（内存/性能）
z1 = x + y  # 非原地加法：会新分配一个张量保存结果；对比 x.add_(y) 会直接改写 x，减少一次分配
x2 = x.clone()  # 克隆一份，保留梯度追踪，重新分配内存
x2.add_(y)  # 原地加法：把结果写回 x2；对比 x2 += y 语义类似，但 add_ 更显式也更常见于性能敏感代码
x3 = x[:, :]  # 全切片：不复制数据，共享内存，保留原有的梯度追踪
x4 = x.detach()  # 分离张量：不复制数据，共享内存，不跟踪梯度
# ----  # 分隔：关闭梯度追踪（推理/评估）
with torch.no_grad():  # 禁用 autograd：节省显存/内存并加速推理；对比 inference_mode 进一步减少开销但限制更多
    z2 = (x @ y).relu()  # 在 no_grad 中做 matmul+ReLU：不会构建计算图；对比训练时通常不包 no_grad
# ----  # 分隔：torch.compile（可用时）
compiled = None  # 先设占位：在不支持 compile 的版本上也能正常运行
if hasattr(torch, 'compile'):  # 兼容性判断：PyTorch 2.x 才有 torch.compile
    f = lambda a, b: (a @ b).relu()  # 定义待编译函数：示例用 matmul+relu；对比写成 def 更易调试，性能目标一致
    compiled = torch.compile(f)  # 编译加速（可能有首次编译开销）；对比 torch.jit.script/trace 是另一套图机制
# ----  # 分隔：一致性与返回值
(z1 - x2).abs().max().item(), z2.shape, compiled is not None  # 检查原地/非原地结果一致，返回 z2 形状与 compile 可用性


### 自动梯度
如果不设置 `requires_grad=True`，则默认不跟踪梯度。注意有两个成员变量：
- `requires_grad`：是否跟踪梯度
- `grad`：梯度值
grad只有发生了反向传播，才会被计算出来，否则为None，而且注意**只有在叶子节点才会被保留**。

我们看个例子: 
$\\ b=2a^Ta \\ c = b^3$


In [ ]:
a = torch.linspace(0,10,6,dtype=torch.float32,device=device,requires_grad=True)
b = a.dot(a) * 2
c = b**3
c.backward()
c,c.grad,b,b.grad,a,a.grad

可以看到，c的梯度和b的梯度都是None，只有叶子节点的a有梯度  

另外，我们调用反向计算时，一般都是在标量上调用，因为如果是在向量上调用，会得到一个矩阵，所以如果一定要在向量上调用，我们一般先sum一下，把向量变成标量，或者backword函数传入一个ones_like这个向量的向量

In [ ]:
a = torch.arange(0,10,2,requires_grad=True,dtype=torch.float32,device=device)
b = a*a
# b.backward() 这样会报错"grad can be implicitly created only for scalar outputs"，因为b不是一个标量
# b.sum().backward()
b.backward(torch.ones_like(b))
a,a.grad,torch.ones_like(b)

### 分离计算
当某个变量之前的计算过程不用考虑，用detach分离出来，detach方法返回的可以看做一个常量，它的梯度永远是 None。

In [ ]:
a.grad.zero_()
b = a*a
u = b.detach()
c = u*a
c.sum().backward()
a.grad == u 


### 自动微分的传染性
对于一个变量a required_grad=True，它和其他变量、常量计算的结果，只要没有with torch.no_grad()，结果就一定是required_grad=True

In [ ]:
a = torch.arange(1,15,3,dtype=torch.float32,requires_grad=True,device=device) ## 自动微分
b = torch.full(a.shape,2,dtype=torch.float32,requires_grad=False,device=device) ## 不自动微分
c = a*b
with torch.no_grad():
    d = a*b
# d.sum().backward() 会报错
c.requires_grad,d.requires_grad # True, False